## Section 1: Operator Overview 
* Using the most recent month

In [1]:
import _report_utils
import _section1_utils as section1
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis.sql import to_snakecase
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

DataTransformerRegistry.enable('default')

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
name = "Bay Area 511 AC Transit Schedule"
organization_name = "Alameda-Contra Costa Transit District"

In [5]:
selected_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = selected_date.year
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

### Map

In [6]:
ac_transit_map = section1.load_operator_map(name)

In [7]:
ac_transit_map.shape

(132, 15)

In [8]:
ac_transit_map.columns

Index(['shape_array_key', 'geometry', 'feed_key', 'schedule_gtfs_dataset_key',
       'Direction', 'route_key', 'service meters', 'service miles',
       'Organization ID', 'Organization', 'Transit Operator',
       'route_long_name', 'route_short_name', 'Route', 'Route ID'],
      dtype='object')

### Operator Profiles

In [9]:
ac_transit_profile = section1.load_operator_profiles(organization_name)

In [10]:
ac_transit_profile

,schedule_gtfs_dataset_key,# routes,# trips,# shapes,# stops,# arrivals,service miles,avg arrivals per stop,# downtown local route types,# local route types,# rapid route types,# coverage route types,Transit Operator,Organization ID,Organization,Date
13,c499f905e33929a641f083dad55c521e,132,5478,346,4733,234582,1459.66,49.56,179,21,92,44,Bay Area 511 AC Transit Schedule,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,2024-04-17


#### Original

In [12]:
op_profiles_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_profiles}.parquet"

op_profiles_df = pd.read_parquet(
    op_profiles_url)

In [13]:
op_profiles_df1 = op_profiles_df.sort_values(by = ['service_date'], ascending = False)

In [16]:
op_profiles_df1.columns

Index(['schedule_gtfs_dataset_key', 'operator_n_routes', 'operator_n_trips',
       'operator_n_shapes', 'operator_n_stops', 'operator_n_arrivals',
       'operator_route_length_miles', 'operator_arrivals_per_stop',
       'n_downtown_local_routes', 'n_local_routes', 'n_rapid_routes',
       'n_coverage_routes', 'name', 'organization_source_record_id',
       'organization_name', 'service_date'],
      dtype='object')

In [19]:
op_profiles_df1.drop_duplicates(subset = ['organization_name'])[['service_date']].describe()

/tmp/ipykernel_1385/18351425.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  op_profiles_df1.drop_duplicates(subset = ['organization_name'])[['service_date']].describe()


,service_date
count,171
unique,11
top,2024-04-17 00:00:00
freq,139
first,2023-03-15 00:00:00
last,2024-04-17 00:00:00


### Operators who don't have profile info from the most recent service_date but have stuff for sched_vp
* City of Torrance	
* Stanislaus Regional Transit Authority

In [22]:
op_profiles2 = op_profiles_df1.drop_duplicates(subset = ['organization_name'])[['organization_name','service_date']]

In [27]:
# op_profiles2.loc[op_profiles2.service_date < '2024-04-17'].sort_values(by = ['organization_name'])

In [29]:
op_profiles_df1.loc[op_profiles_df1.organization_name.isin(['City of Torrance','Stanislaus Regional Transit Authority'])][['organization_name','service_date']]

,organization_name,service_date
1595,Stanislaus Regional Transit Authority,2024-03-13
348,City of Torrance,2024-02-14
1594,Stanislaus Regional Transit Authority,2024-01-17
347,City of Torrance,2024-01-17
1593,Stanislaus Regional Transit Authority,2023-12-13
346,City of Torrance,2023-12-13
345,City of Torrance,2023-11-15
1592,Stanislaus Regional Transit Authority,2023-11-15
344,City of Torrance,2023-10-11
318,Stanislaus Regional Transit Authority,2023-10-11
